# Importação de base de dados

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
import sklearn.metrics as metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, InputLayer

2021-09-06 01:53:07.062368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-06 01:53:07.062394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
transact_train_basic = pd.read_csv('TRN_X.csv')
transact_test_basic = pd.read_csv('TST_X.csv')

transact_train_basic_y = pd.read_csv('TRN_Y.csv')
transact_test_basic_y = pd.read_csv('TST_Y.csv')


In [3]:
# transact_train_basic.head()

transact_train_basic_y.head()

,1
0,1
1,1
2,0
3,1
4,0


# Classificador Random Forest

In [4]:
rc_basic = RandomForestClassifier(n_estimators=100, max_features=10)
rc_basic.fit(transact_train_basic, transact_train_basic_y.values)

y_predict = rc_basic.predict(transact_test_basic)

np.sqrt(mean_squared_error(transact_test_basic_y, y_predict))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').